In [2]:
pip install numpy matplotlib opencv-python tensorflow scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

train = train_datagen.flow_from_directory("data1/train", target_size=(150,150), class_mode='binary')
val = val_datagen.flow_from_directory("data1/val", target_size=(150,150), class_mode='binary')
test = val_datagen.flow_from_directory("data1/test", target_size=(150,150), class_mode='binary')


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import Input  # ✅ this line fixes the error

model = Sequential([
    Input(shape=(150, 150, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])


In [5]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train, validation_data=val, epochs=10)


C:\Users\Subhajit Lewra\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 268s 2s/step - accuracy: 0.7538 - loss: 0.5488 - val_accuracy: 0.6875 - val_loss: 1.0557
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 162s 994ms/step - accuracy: 0.8820 - loss: 0.2755 - val_accuracy: 0.6875 - val_loss: 0.7252
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 199s 1s/step - accuracy: 0.9101 - loss: 0.2254 - val_accuracy: 0.7500 - val_loss: 0.4188
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 191s 1s/step - accuracy: 0.9195 - loss: 0.2049 - val_accuracy: 0.6875 - val_loss: 0.7947
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 151s 927ms/step - accuracy: 0.9272 - loss: 0.1962 - val_accuracy: 0.6875 - val_loss: 1.0031
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 150s 915ms/step - accuracy: 0.9264 - loss: 0.1880 - val_accuracy: 0.6250 - val_loss: 1.1736
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 151s 923ms/step - accuracy: 0.9272 - loss: 0.1875 - val_accuracy: 0.6875 - val_loss: 1.1178
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 150s 917ms/step - accuracy: 0.9299 - loss: 0.

In [6]:
loss, accuracy = model.evaluate(test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


20/20 ━━━━━━━━━━━━━━━━━━━━ 18s 917ms/step - accuracy: 0.9052 - loss: 0.3049
Test Accuracy: 90.22%


In [7]:
model.save("pneumonia_detector.h5")


In [15]:
import numpy as np
from tensorflow.keras.preprocessing import image

img = image.load_img("data1/val/NORMAL/NORMAL2-IM-1427-0001.jpeg", target_size=(150,150))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)
print("Pneumonia" if prediction[0][0] > 0.5 else "Normal")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Pneumonia
